In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg, when, count, lit, udf
from pyspark.sql.window import Window

In [2]:
# Inicializando a sessão Spark
spark = SparkSession.builder \
    .appName("PySpark Exercises") \
    .getOrCreate()

24/07/20 18:53:12 WARN Utils: Your hostname, victor-aspirea31558 resolves to a loopback address: 127.0.1.1; using 192.168.0.3 instead (on interface wlp0s20f3)
24/07/20 18:53:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/20 18:53:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Definindo o nível de log para ERROR
spark.sparkContext.setLogLevel("ERROR")

In [4]:
# Carregar os dados de exemplo
customers = spark.read.csv("/home/vicrrs/Projetos_github/exercicios_python/ExPyspark/Ex001/data/customers.csv", header=True, inferSchema=True)
orders = spark.read.csv("/home/vicrrs/Projetos_github/exercicios_python/ExPyspark/Ex001/data/orders.csv", header=True, inferSchema=True)
products = spark.read.csv("/home/vicrrs/Projetos_github/exercicios_python/ExPyspark/Ex001/data/products.csv", header=True, inferSchema=True)

### Leitura dos dados

In [5]:
customers.show()

+-----------+-------------+---+-----------+
|customer_id|customer_name|age|       city|
+-----------+-------------+---+-----------+
|          1|     John Doe| 28|   New York|
|          2|   Jane Smith| 34|Los Angeles|
|          3|Emily Johnson| 21|    Chicago|
|          4|Michael Brown| 45|    Houston|
|          5|  Linda Davis| 39|    Phoenix|
+-----------+-------------+---+-----------+



In [6]:
orders.show()

+--------+-----------+----------+--------+-----+
|order_id|customer_id|   product|quantity|price|
+--------+-----------+----------+--------+-----+
|     101|          1|    Laptop|       1| 1000|
|     102|          2|Smartphone|       2|  500|
|     103|          1|    Tablet|       3|  200|
|     104|          3|Headphones|       1|  150|
|     105|          4|   Monitor|       2|  300|
+--------+-----------+----------+--------+-----+



In [7]:
products.show()

+----------+------------+-----------+-----+
|product_id|product_name|   category|price|
+----------+------------+-----------+-----+
|         1|      Laptop|Electronics| 1000|
|         2|  Smartphone|Electronics|  500|
|         3|      Tablet|Electronics|  200|
|         4|  Headphones|Accessories|  150|
|         5|     Monitor|Electronics|  300|
+----------+------------+-----------+-----+



### Exibir Schemas

In [8]:
customers.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- city: string (nullable = true)



In [9]:
orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- price: integer (nullable = true)



In [10]:
products.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- price: integer (nullable = true)



### Contar Registros

In [11]:
customers.count()

5

In [12]:
orders.count()

5

In [13]:
products.count()

5

### Selecionar Colunas

In [14]:
customers.select("customer_id", "customer_name").show()

+-----------+-------------+
|customer_id|customer_name|
+-----------+-------------+
|          1|     John Doe|
|          2|   Jane Smith|
|          3|Emily Johnson|
|          4|Michael Brown|
|          5|  Linda Davis|
+-----------+-------------+



In [15]:
orders.select("customer_id", "product").show()

+-----------+----------+
|customer_id|   product|
+-----------+----------+
|          1|    Laptop|
|          2|Smartphone|
|          1|    Tablet|
|          3|Headphones|
|          4|   Monitor|
+-----------+----------+



In [16]:
products.select("product_name", "price").show()

+------------+-----+
|product_name|price|
+------------+-----+
|      Laptop| 1000|
|  Smartphone|  500|
|      Tablet|  200|
|  Headphones|  150|
|     Monitor|  300|
+------------+-----+



### Filtrar Dados
    * Filtre os clientes com idade maior que 30 anos.

In [17]:
customers.filter(customers.age > 30).show()

+-----------+-------------+---+-----------+
|customer_id|customer_name|age|       city|
+-----------+-------------+---+-----------+
|          2|   Jane Smith| 34|Los Angeles|
|          4|Michael Brown| 45|    Houston|
|          5|  Linda Davis| 39|    Phoenix|
+-----------+-------------+---+-----------+



In [18]:
orders.filter(orders.quantity > 2).show()

+--------+-----------+-------+--------+-----+
|order_id|customer_id|product|quantity|price|
+--------+-----------+-------+--------+-----+
|     103|          1| Tablet|       3|  200|
+--------+-----------+-------+--------+-----+



In [19]:
products.filter(products.price >= 200).show()

+----------+------------+-----------+-----+
|product_id|product_name|   category|price|
+----------+------------+-----------+-----+
|         1|      Laptop|Electronics| 1000|
|         2|  Smartphone|Electronics|  500|
|         3|      Tablet|Electronics|  200|
|         5|     Monitor|Electronics|  300|
+----------+------------+-----------+-----+



In [20]:
products.filter((products.price >= 180) | (products.price < 100)).show()

+----------+------------+-----------+-----+
|product_id|product_name|   category|price|
+----------+------------+-----------+-----+
|         1|      Laptop|Electronics| 1000|
|         2|  Smartphone|Electronics|  500|
|         3|      Tablet|Electronics|  200|
|         5|     Monitor|Electronics|  300|
+----------+------------+-----------+-----+



In [21]:
products.filter((products.price >= 100) & (products.price < 400)).show()

+----------+------------+-----------+-----+
|product_id|product_name|   category|price|
+----------+------------+-----------+-----+
|         3|      Tablet|Electronics|  200|
|         4|  Headphones|Accessories|  150|
|         5|     Monitor|Electronics|  300|
+----------+------------+-----------+-----+



### Ordenar dados
    * Ordene o DataFrame de clientes por idade em ordem decrescente.

In [22]:
customers.orderBy(col("age").desc()).show()

+-----------+-------------+---+-----------+
|customer_id|customer_name|age|       city|
+-----------+-------------+---+-----------+
|          4|Michael Brown| 45|    Houston|
|          5|  Linda Davis| 39|    Phoenix|
|          2|   Jane Smith| 34|Los Angeles|
|          1|     John Doe| 28|   New York|
|          3|Emily Johnson| 21|    Chicago|
+-----------+-------------+---+-----------+



### Agrupar Dados
    * Agrupe os pedidos por customer_id e calcule a soma das quantidades.

In [23]:
orders.groupBy("customer_id").agg(sum("quantity").alias("total_quantity")).show()

+-----------+--------------+
|customer_id|total_quantity|
+-----------+--------------+
|          1|             4|
|          3|             1|
|          4|             2|
|          2|             2|
+-----------+--------------+



### Joins
    * Realize um inner join entre os DataFrames de clientes e pedidos com base no customer_id.

In [24]:
customers.join(orders, "customer_id").show()

+-----------+-------------+---+-----------+--------+----------+--------+-----+
|customer_id|customer_name|age|       city|order_id|   product|quantity|price|
+-----------+-------------+---+-----------+--------+----------+--------+-----+
|          1|     John Doe| 28|   New York|     103|    Tablet|       3|  200|
|          1|     John Doe| 28|   New York|     101|    Laptop|       1| 1000|
|          2|   Jane Smith| 34|Los Angeles|     102|Smartphone|       2|  500|
|          3|Emily Johnson| 21|    Chicago|     104|Headphones|       1|  150|
|          4|Michael Brown| 45|    Houston|     105|   Monitor|       2|  300|
+-----------+-------------+---+-----------+--------+----------+--------+-----+



### Left Join

    * Realize um left join entre os DataFrames de pedidos e produtos com base no product.

In [26]:
orders.join(products, orders.product == products.product_name, "left").show()

+--------+-----------+----------+--------+-----+----------+------------+-----------+-----+
|order_id|customer_id|   product|quantity|price|product_id|product_name|   category|price|
+--------+-----------+----------+--------+-----+----------+------------+-----------+-----+
|     101|          1|    Laptop|       1| 1000|         1|      Laptop|Electronics| 1000|
|     102|          2|Smartphone|       2|  500|         2|  Smartphone|Electronics|  500|
|     103|          1|    Tablet|       3|  200|         3|      Tablet|Electronics|  200|
|     104|          3|Headphones|       1|  150|         4|  Headphones|Accessories|  150|
|     105|          4|   Monitor|       2|  300|         5|     Monitor|Electronics|  300|
+--------+-----------+----------+--------+-----+----------+------------+-----------+-----+



### Right Join

    * Realize um right join entre os DataFrames de pedidos e clientes com base no customer_id.

In [28]:
orders.join(customers, "customer_id", "right").show()

+-----------+--------+----------+--------+-----+-------------+---+-----------+
|customer_id|order_id|   product|quantity|price|customer_name|age|       city|
+-----------+--------+----------+--------+-----+-------------+---+-----------+
|          1|     103|    Tablet|       3|  200|     John Doe| 28|   New York|
|          1|     101|    Laptop|       1| 1000|     John Doe| 28|   New York|
|          2|     102|Smartphone|       2|  500|   Jane Smith| 34|Los Angeles|
|          3|     104|Headphones|       1|  150|Emily Johnson| 21|    Chicago|
|          4|     105|   Monitor|       2|  300|Michael Brown| 45|    Houston|
|          5|    NULL|      NULL|    NULL| NULL|  Linda Davis| 39|    Phoenix|
+-----------+--------+----------+--------+-----+-------------+---+-----------+



### Full Outer Join

    * Realize um full outer join entre os DataFrames de pedidos e produtos com base no product.

In [29]:
orders.join(products, orders.product == products.product_name, "outer").show()

+--------+-----------+----------+--------+-----+----------+------------+-----------+-----+
|order_id|customer_id|   product|quantity|price|product_id|product_name|   category|price|
+--------+-----------+----------+--------+-----+----------+------------+-----------+-----+
|     104|          3|Headphones|       1|  150|         4|  Headphones|Accessories|  150|
|     101|          1|    Laptop|       1| 1000|         1|      Laptop|Electronics| 1000|
|     105|          4|   Monitor|       2|  300|         5|     Monitor|Electronics|  300|
|     102|          2|Smartphone|       2|  500|         2|  Smartphone|Electronics|  500|
|     103|          1|    Tablet|       3|  200|         3|      Tablet|Electronics|  200|
+--------+-----------+----------+--------+-----+----------+------------+-----------+-----+



### Cruzamento de Dados

    * Realize um cross join entre os DataFrames de clientes e produtos.

In [30]:
customers.crossJoin(products).show()

+-----------+-------------+---+-----------+----------+------------+-----------+-----+
|customer_id|customer_name|age|       city|product_id|product_name|   category|price|
+-----------+-------------+---+-----------+----------+------------+-----------+-----+
|          1|     John Doe| 28|   New York|         1|      Laptop|Electronics| 1000|
|          2|   Jane Smith| 34|Los Angeles|         1|      Laptop|Electronics| 1000|
|          3|Emily Johnson| 21|    Chicago|         1|      Laptop|Electronics| 1000|
|          4|Michael Brown| 45|    Houston|         1|      Laptop|Electronics| 1000|
|          5|  Linda Davis| 39|    Phoenix|         1|      Laptop|Electronics| 1000|
|          1|     John Doe| 28|   New York|         2|  Smartphone|Electronics|  500|
|          2|   Jane Smith| 34|Los Angeles|         2|  Smartphone|Electronics|  500|
|          3|Emily Johnson| 21|    Chicago|         2|  Smartphone|Electronics|  500|
|          4|Michael Brown| 45|    Houston|         2|

### Criação de Colunas

    * Crie uma nova coluna total_price no DataFrame de pedidos calculando a multiplicação de quantity e price.

In [31]:
orders.withColumn("total_price", orders.quantity * orders.price).show()

+--------+-----------+----------+--------+-----+-----------+
|order_id|customer_id|   product|quantity|price|total_price|
+--------+-----------+----------+--------+-----+-----------+
|     101|          1|    Laptop|       1| 1000|       1000|
|     102|          2|Smartphone|       2|  500|       1000|
|     103|          1|    Tablet|       3|  200|        600|
|     104|          3|Headphones|       1|  150|        150|
|     105|          4|   Monitor|       2|  300|        600|
+--------+-----------+----------+--------+-----+-----------+



### Conversão de Tipos

    * Converta a coluna age do DataFrame de clientes para o tipo String.

In [32]:
customers.withColumn("age", col("age").cast("string")).printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- city: string (nullable = true)

